In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
import torch
import torch.special as sp
from torch.utils.data import Dataset
from datasets import load_dataset
from energizer.data import ActiveDataset, ActiveDataModule
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [3]:
ds = load_dataset("pietrolesci/ag_news", name="concat")

Reusing dataset ag_news (/Users/pietrolesci/.cache/huggingface/datasets/pietrolesci___ag_news/concat/1.0.0/5ee6e111adc7a901ca734b79fbebff09d9dba91722387a794efff8d9c178a6a3)
100%|██████████| 2/2 [00:00<00:00, 136.63it/s]


In [4]:
ds = ds.map(lambda ex: tokenizer(ex["text"], return_token_type_ids=False), batched=True)

Loading cached processed dataset at /Users/pietrolesci/.cache/huggingface/datasets/pietrolesci___ag_news/concat/1.0.0/5ee6e111adc7a901ca734b79fbebff09d9dba91722387a794efff8d9c178a6a3/cache-b54342625c18b8be.arrow
Loading cached processed dataset at /Users/pietrolesci/.cache/huggingface/datasets/pietrolesci___ag_news/concat/1.0.0/5ee6e111adc7a901ca734b79fbebff09d9dba91722387a794efff8d9c178a6a3/cache-6c0037a17af54a3c.arrow


In [5]:
ds = ds.with_format(columns=["input_ids", "attention_mask"])

In [19]:
dm = ActiveDataModule(
    num_classes=2,
    train_dataset=ds["train"],
    # val_dataset=ds["test"],
    initial_labels=np.random.choice(list(range(len(ds["train"]))), size=10, replace=False).tolist(),
    min_steps_per_epoch=10,
    val_split=0.3,
    test_dataset=ds["test"],
    batch_size=1,
    seed=42,
    collate_fn=DataCollatorWithPadding(tokenizer),
)

Validation dataset is not specfied and `val_split == 0.3`, therefore at each training step 0.3 of the labelled data will be added to the validation dataset.


In [20]:
next(iter(dm.train_dataloader()))

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'input_ids': tensor([[  101,  6034, 12702,  5428,  4523,  2202,  1996,  3684,  2887,  6529,
          2360,  2027,  2031, 26176,  2028,  1997,  1996,  2197, 28750,  1997,
         20681,  2974,  1010, 10058,  2073,  4816,  5733,  2031,  2000,  2448,
          2012,  2152,  2373,  1010,  1999,  9205,  3684,  2030,  2012, 12765,
         21656,  1997,  8249,  1012,   102]])}

In [21]:
next(iter(dm.val_dataloader()))

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'input_ids': tensor([[  101, 11418, 10299,  1996,  2395,  1999,  1053,  2475,  1010, 11139,
          1996,  3347,  2470,  1999,  4367,  5183,  1012,  1010,  9338,  1997,
          1996,  2759, 25935,  1041,  1011,  5653,  5080,  1010,  5632,  1037,
          2844,  2117,  4284,  1998,  2039,  5669,  2049, 10908,  9432,  2005,
          1996,  6893,  1997,  2049, 10807,  2095,  1012,   102]])}

In [22]:
next(iter(dm.test_dataloader()))

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]]), 'input_ids': tensor([[  101, 10069,  2005,  1056,  1050, 11550,  2044,  7566,  9209,  5052,
          3667,  2012,  6769,  2047,  8095,  2360,  2027,  2024,  1005,  9364,
          1005,  2044,  7566,  2007, 16654,  6687,  3813,  2976,  9587, 24848,
          1012,   102]])}